# Chapter 10: Testing
\n**Note:** This notebook will install Rust in your Colab environment. Run the setup cell first!\n
Learn how to write and run tests in Rust.

In [ ]:
%%bash
# Install Rust in Colab (run this cell first!)
curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
source $HOME/.cargo/env
rustc --version

## How to Write Tests

Tests in Rust are functions annotated with the `#[test]` attribute.

In [ ]:
#[cfg(test)]
mod tests {
    #[test]
    fn it_works() {
        let result = 2 + 2;
        assert_eq!(result, 4);
    }
    
    #[test]
    fn another() {
        assert!(true);
    }
}

## Test Assertions

Common assertion macros:
- `assert!()` - checks if a boolean is true
- `assert_eq!()` - checks equality
- `assert_ne!()` - checks inequality

In [ ]:
#[derive(Debug, PartialEq)]
struct Rectangle {
    width: u32,
    height: u32,
}

impl Rectangle {
    fn can_hold(&self, other: &Rectangle) -> bool {
        self.width > other.width && self.height > other.height
    }
}

#[cfg(test)]
mod tests {
    use super::*;
    
    #[test]
    fn larger_can_hold_smaller() {
        let larger = Rectangle { width: 8, height: 7 };
        let smaller = Rectangle { width: 5, height: 1 };
        
        assert!(larger.can_hold(&smaller));
    }
    
    #[test]
    fn smaller_cannot_hold_larger() {
        let larger = Rectangle { width: 8, height: 7 };
        let smaller = Rectangle { width: 5, height: 1 };
        
        assert!(!smaller.can_hold(&larger));
    }
}

## Custom Failure Messages

In [ ]:
pub fn greeting(name: &str) -> String {
    format!("Hello {}!", name)
}

#[cfg(test)]
mod tests {
    use super::*;
    
    #[test]
    fn greeting_contains_name() {
        let result = greeting("Carol");
        assert!(
            result.contains("Carol"),
            "Greeting did not contain name, value was `{}`",
            result
        );
    }
}

## Testing for Panics

In [ ]:
pub struct Guess {
    value: i32,
}

impl Guess {
    pub fn new(value: i32) -> Guess {
        if value < 1 || value > 100 {
            panic!("Guess value must be between 1 and 100, got {}.", value);
        }
        
        Guess { value }
    }
}

#[cfg(test)]
mod tests {
    use super::*;
    
    #[test]
    #[should_panic(expected = "Guess value must be between 1 and 100")]
    fn greater_than_100() {
        Guess::new(200);
    }
}

## Using Result<T, E> in Tests

In [ ]:
#[cfg(test)]
mod tests {
    #[test]
    fn it_works() -> Result<(), String> {
        if 2 + 2 == 4 {
            Ok(())
        } else {
            Err(String::from("two plus two does not equal four"))
        }
    }
}

## Controlling Test Execution

Run tests with:
```bash
cargo test
```

Useful flags:
- `cargo test -- --test-threads=1` - Run tests serially
- `cargo test -- --show-output` - Show println! output
- `cargo test test_name` - Run specific test
- `cargo test -- --ignored` - Run ignored tests

## Ignoring Tests

In [ ]:
#[cfg(test)]
mod tests {
    #[test]
    fn it_works() {
        assert_eq!(2 + 2, 4);
    }
    
    #[test]
    #[ignore]
    fn expensive_test() {
        // This test takes a long time
    }
}

## Test Organization

### Unit Tests
- Test one module in isolation
- Live in `src/` next to the code being tested
- Can test private functions

In [ ]:
pub fn add_two(a: i32) -> i32 {
    internal_adder(a, 2)
}

fn internal_adder(a: i32, b: i32) -> i32 {
    a + b
}

#[cfg(test)]
mod tests {
    use super::*;
    
    #[test]
    fn internal() {
        assert_eq!(4, internal_adder(2, 2));
    }
}

## Integration Tests

Integration tests live in the `tests/` directory and test your library's public API.

```
adder
├── Cargo.lock
├── Cargo.toml
├── src
│   └── lib.rs
└── tests
    └── integration_test.rs
```

In [ ]:
// tests/integration_test.rs
use adder;

#[test]
fn it_adds_two() {
    assert_eq!(4, adder::add_two(2));
}

// Run with: cargo test --test integration_test

## Submodules in Integration Tests

In [ ]:
// tests/common/mod.rs
pub fn setup() {
    // Setup code here
}

// tests/integration_test.rs
mod common;

#[test]
fn it_adds_two() {
    common::setup();
    // Test code
}

## Documentation Tests

Code examples in documentation comments are automatically tested!

In [ ]:
/// Adds one to the number given.
///
/// # Examples
///
/// ```
/// let arg = 5;
/// let answer = my_crate::add_one(arg);
///
/// assert_eq!(6, answer);
/// ```
pub fn add_one(x: i32) -> i32 {
    x + 1
}

## Exercises

1. Write tests for a calculator with add, subtract, multiply, divide
2. Test that divide panics when dividing by zero
3. Create integration tests for a library
4. Write a test that uses Result<T, E>

In [ ]:
// Exercise: Calculator tests
pub struct Calculator;

impl Calculator {
    pub fn add(a: i32, b: i32) -> i32 {
        // Your implementation
        0
    }
    
    pub fn divide(a: i32, b: i32) -> i32 {
        if b == 0 {
            panic!("Division by zero");
        }
        a / b
    }
}

#[cfg(test)]
mod tests {
    use super::*;
    
    #[test]
    fn test_add() {
        // Your test here
    }
    
    #[test]
    #[should_panic(expected = "Division by zero")]
    fn test_divide_by_zero() {
        // Your test here
    }
}

## Key Takeaways

- Use `#[test]` attribute to mark test functions
- `assert!`, `assert_eq!`, and `assert_ne!` are core assertions
- Use `#[should_panic]` to test panic conditions
- Unit tests live with the code in `#[cfg(test)]` modules
- Integration tests live in the `tests/` directory
- Documentation examples are automatically tested
- Run tests with `cargo test`